## Qwen 0.6B Model

### Install SUMMARISER MODEL

In [ ]:
!pip3 install bert-extractive-summarizer

Necessary Libraries

In [1]:
import ffmpeg
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import re
import os
from tqdm import tqdm
import csv

# Paths
VIDEO_DIR = "Data/Videos"             # Folder containing input video files
AUDIO_DIR = "Data/Audios"             # Folder to store extracted audio files
CHUNK_DIR = "Data/Audio-Chunks"       # Folder to save audio chunks after VAD

# Create output directories if they don't exist
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(CHUNK_DIR, exist_ok=True)

# Audio processing
TARGET_SAMPLE_RATE = 16000  # or 32000 Hz depending on your use case

# VAD settings
MIN_CHUNK_DURATION_SEC = 30  # Minimum duration for an audio chunk
USE_ONNX_MODEL = False      # Set True to use ONNX version of Silero VAD

from silero_vad import (
    load_silero_vad, read_audio, get_speech_timestamps, 
    save_audio, VADIterator
)

faiss_index = faiss.read_index("Data/sentence_embeddings.index")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Model for Semantic Search

In [ ]:
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

### Summarisation

In [2]:
from summarizer import Summarizer

# Initialize summarizer
summarizer = Summarizer()

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Video to Audio Conversion

In [ ]:
# Loop through all .mp4 files in the input folder
for filename in os.listdir(VIDEO_DIR):
    if filename.endswith(".mp4"):
        input_path = os.path.join(VIDEO_DIR, filename)
        output_path = os.path.join(AUDIO_DIR, filename.replace(".mp4", ".wav"))

        print(f"Processing: {input_path} -> {output_path}")
        
        # Extract audio
        input_video = ffmpeg.input(input_path)
        output_audio = ffmpeg.output(input_video.audio, output_path, ac=1, ar=TARGET_SAMPLE_RATE)
        ffmpeg.run(output_audio, overwrite_output=True)
        
        # Probe the generated audio file for details
        audio_info = ffmpeg.probe(output_path, v="error", select_streams="a", show_entries="stream=codec_name,codec_type,sample_rate,channels,bit_rate,duration")
        
        codec_name = audio_info['streams'][0]['codec_name']
        sample_rate = int(audio_info['streams'][0]['sample_rate'])
        channels = int(audio_info['streams'][0]['channels'])
        bit_rate = audio_info['streams'][0].get('bit_rate', 'N/A')
        duration_sec = float(audio_info['streams'][0]['duration'])
        duration_ms = duration_sec * 1000
        
        print(f"Audio extracted: {output_path}")
        # print(f"Codec: {codec_name}, Sample Rate: {sample_rate} Hz, Channels: {channels}, Bit Rate: {bit_rate}, Duration: {duration_ms} ms\n")

print("Video to Audio converted successfully!.")

Voice Activity Detection Algorithm on Audio Files - Converting into Smaller Chunks

In [ ]:
# Load Silero VAD model
model = load_silero_vad(onnx=USE_ONNX_MODEL)

def process_audio_file(audio_path, output_chunk_dir):
    """Process an audio file, split it into chunks, and save them."""
    wav = read_audio(audio_path, sampling_rate=TARGET_SAMPLE_RATE)
    speech_timestamps = get_speech_timestamps(
        wav, model, sampling_rate=TARGET_SAMPLE_RATE, return_seconds=True
    )
    
    # Format timestamps to 4 decimal places
    for segment in speech_timestamps:
        segment['start'] = float(f"{segment['start']:.4f}")
        segment['end'] = float(f"{segment['end']:.4f}")
    
    vad_iterator = VADIterator(model, sampling_rate=TARGET_SAMPLE_RATE)
    chunks = []
    current_chunk_start = 0
    
    for segment in speech_timestamps:
        start, end = segment['start'], segment['end']
        if (end - current_chunk_start) >= MIN_CHUNK_DURATION_SEC:
            chunk_wav = wav[int(current_chunk_start * TARGET_SAMPLE_RATE):int(end * TARGET_SAMPLE_RATE)]
            chunk_path = os.path.join(output_chunk_dir, f"{len(chunks) + 1}.wav")
            save_audio(chunk_path, chunk_wav, sampling_rate=TARGET_SAMPLE_RATE)
            chunks.append((current_chunk_start, end, chunk_wav))
            current_chunk_start = end
    
    # Save the last chunk if necessary
    if current_chunk_start < speech_timestamps[-1]['end']:
        chunk_wav = wav[int(current_chunk_start * TARGET_SAMPLE_RATE):]
        chunk_path = os.path.join(output_chunk_dir, f"{len(chunks) + 1}.wav")
        save_audio(chunk_path, chunk_wav, sampling_rate=TARGET_SAMPLE_RATE)
        chunks.append((current_chunk_start, speech_timestamps[-1]['end'], chunk_wav))
    
    vad_iterator.reset_states()
    print(f"Processed {audio_path}, saved chunks in {output_chunk_dir}")

def process_all_audio_files():
    """Process all .wav files in the main audio folder and save their chunks."""
    if not os.path.exists(AUDIO_DIR):
        print(f"Audio folder '{AUDIO_DIR}' does not exist.")
        return
    
    for file_name in sorted(os.listdir(AUDIO_DIR)):
        if file_name.endswith(".wav"):
            audio_path = os.path.join(AUDIO_DIR, file_name)
            audio_id = os.path.splitext(file_name)[0]  # Extract the number without extension
            output_chunk_dir = os.path.join(CHUNK_DIR, audio_id)
            process_audio_file(audio_path, output_chunk_dir)

# Process VAD on Audio Files.
process_all_audio_files()


## SRT to Sentences generation

In [ ]:
# Function to extract combined sentences with timestamps from .srt file
def extract_sentences_from_srt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    sentences = []
    timestamps = []
    current_sentence = ""
    current_timestamp = ""

    for line in lines:
        line = line.strip()

        # Check for timestamp lines
        timestamp_match = re.match(r'(\d{2}:\d{2}:\d{2}[.,]\d{3}) --> (\d{2}:\d{2}:\d{2}[.,]\d{3})', line)
        if timestamp_match:
            current_timestamp = timestamp_match.group(1).replace(',', '.') + ' --> ' + timestamp_match.group(2).replace(',', '.')
            continue

        # Skip empty lines and cue identifiers
        if not line or line.isdigit():
            continue

        # Add line to current sentence
        current_sentence += " " + line if current_sentence else line

        # If sentence ends, save it
        if re.search(r'[.!?]$', line):
            sentences.append(current_sentence.strip())
            timestamps.append(current_timestamp)
            current_sentence = ""
            current_timestamp = ""

    return sentences, timestamps

# Directory containing SRT files
srt_directory = 'Data/SRT-Files'

# Initialize lists for all sentences, timestamps, and filenames
all_sentences = []
all_timestamps = []
all_filenames = []

# Process all SRT files with tqdm
srt_files = [f for f in os.listdir(srt_directory) if f.endswith('.srt')]
for file_name in tqdm(srt_files, desc="Processing SRT files"):
    file_path = os.path.join(srt_directory, file_name)
    sentences, timestamps = extract_sentences_from_srt(file_path)
    all_sentences.extend(sentences)
    all_timestamps.extend(timestamps)
    all_filenames.extend([file_name] * len(sentences))  # associate each sentence with its file

print("Encoding sentences into embeddings...")
# Uncomment for all miniLM and all mpnet.
# sentence_embeddings = np.array(
#     model.encode(all_sentences) 
# ).astype('float32')

sentence_embeddings = model.encode(all_sentences)

# Create FAISS index (use Inner Product for cosine similarity)
embedding_dimension = sentence_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dimension)
faiss_index.add(sentence_embeddings)

# Save FAISS index
faiss.write_index(faiss_index, "Data/sentence_embeddings.index")

# Save metadata to file
metadata_file = 'Data/srt-embedding-metadata.tsv'
with open(metadata_file, 'w', encoding='utf-8') as file:
    file.write("filename\ttimestamp\tsentence\n")
    for fname, timestamp, sentence in zip(all_filenames, all_timestamps, all_sentences):
        clean_sentence = sentence.replace('\t', ' ').replace('\n', ' ')
        file.write(f"{fname}\t{timestamp}\t{clean_sentence}\n")

# Save sentences to a text file
sentences_file = 'Data/sentences.txt'
with open(sentences_file, 'w', encoding='utf-8') as file:
    for sentence in all_sentences:
        file.write(sentence.strip().replace('\n', ' ') + '\n')

# Summary
print(f"\nEmbeddings created for {len(all_sentences)} sentences from {len(srt_files)} SRT files.")
print("FAISS index saved as 'Data/sentence_embeddings.index'")
print(f"Metadata saved as '{metadata_file}'")
print(f"Sentences saved as '{sentences_file}'")

## Finding Related Sentences to a Question

In [ ]:
import pickle
# Load sentences
with open('Data/sentences.txt', 'r') as file:
    lecture_sentences = [line.strip() for line in file if line.strip()]

# Group sentences (sliding window)
window_size = 3
grouped_sentences = [
    " ".join(lecture_sentences[i:i + window_size])
    for i in range(len(lecture_sentences) - window_size + 1)
]

# Encode grouped sentences
group_embeddings = model.encode(grouped_sentences)
faiss_index = faiss.IndexFlatL2(group_embeddings.shape[1])
faiss_index.add(group_embeddings)

faiss.write_index(faiss_index, "Data/qwen-0.6B-grouped-sentences-embeddings.idx")

# Save grouped sentences and metadata (optional, for lookup)
with open("Data/grouped_sentences.pkl", "wb") as f:
    pickle.dump(grouped_sentences, f)

### Grouping Sentences N-gram technique

In [ ]:
# Load grouped sentences
with open("Data/grouped_sentences.pkl", "rb") as f:
    grouped_sentences = pickle.load(f)

# Load metadata
metadata_list = []
with open("Data/srt-embedding-metadata.tsv", "r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter='\t')
    header = next(reader)
    for row in reader:
        filename, timestamp, sentence = row
        metadata_list.append((filename, timestamp, sentence))

# Create mapping: grouped_sent -> (filename, start_time, end_time)
grouped_sent_to_metadata = {}

for grouped_sent in grouped_sentences:
    matched_metadata = [(fn, ts) for fn, ts, sent in metadata_list if sent in grouped_sent]

    if matched_metadata:
        # Since your grouped sentences come from **one file only**, pick first filename
        filename_set = set(fn for fn, ts in matched_metadata)
        filename = filename_set.pop()  # should be only one

        # Combine timestamps
        timestamps = [ts for fn, ts in matched_metadata]
        start_time = timestamps[0]
        end_time = timestamps[-1]
    else:
        filename = "Unknown"
        start_time = "Unknown"
        end_time = "Unknown"

    grouped_sent_to_metadata[grouped_sent] = (filename, start_time, end_time)

# Save mapping
with open("Data/grouped_sent_to_metadata.pkl", "wb") as f:
    pickle.dump(grouped_sent_to_metadata, f)

print("Mapping file created successfully!")


### QA

In [ ]:
import pickle
import faiss
import numpy as np

# Load FAISS index
faiss_index = faiss.read_index("Data/qwen-0.6B-grouped-sentences-embeddings.idx")

# Load grouped sentences
with open("Data/grouped_sentences.pkl", "rb") as f:
    grouped_sentences = pickle.load(f)

# Load precomputed metadata mapping
with open("Data/grouped_sent_to_metadata.pkl", "rb") as f:
    grouped_sent_to_metadata = pickle.load(f)

# Encode question
student_question = input("Enter your question: ")
question_embedding = model.encode(student_question, prompt_name="query")
if question_embedding.ndim == 1:
    question_embedding = np.expand_dims(question_embedding, axis=0)

# Search top-10 results
distances, indices = faiss_index.search(question_embedding, 10)

# Retrieve grouped sentences + metadata directly
related_results = []
for idx in indices[0]:
    grouped_sent = grouped_sentences[idx]
    # Get filename and timestamps from precomputed mapping
    filename, start_time, end_time = grouped_sent_to_metadata.get(
        grouped_sent, ("Unknown", "Unknown", "Unknown")
    )
    timestamp_range = f"{start_time} → {end_time}"
    related_results.append((filename, timestamp_range, grouped_sent))

# Print Related Sentences
print("Question:", student_question)
print("\nTop Related Sentences with Metadata:")
for filename, timestamp, sent in related_results:
    print(f"- [{filename} | {timestamp}] {sent}\n")

Question: What are Decision Trees?

Top Related Sentences with Metadata:
- [16.srt | 00:12:20.530 --> 00:12:21.400 → 00:04:38.839 --> 00:04:40.349] So let's have a look. So what is a decision tree? Let's take an example.

- [2.srt | 00:00:44.700 --> 00:00:46.929 → 00:04:38.839 --> 00:04:40.349] So what is a decision tree? Let's take an example. These are the photos of two different kinds of mushroom and one of them is edible and the other one is deadly poisonous.

- [14.srt | 00:08:37.470 --> 00:08:37.810 → 00:09:34.810 --> 00:09:35.080] All right. Hey everyone. In this video, we're going to talk about decision trees.

- [17.srt | 00:00:32.740 --> 00:00:33.329 → 00:01:25.299 --> 00:01:31.549] The decision tree may look like this. Let's say we have different samples of mushroom data and then from this first node it's asking some criteria whether it's large or not. So let's say this one is large and then classify to large equals yes.

- [16.srt | 00:00:05.129 --> 00:00:10.500 → 00:00:19.

### Summarisation

In [ ]:
# Combine all top-K grouped sentences into one text
all_text = " ".join([sent for _, _, sent in related_results])
print(all_text)

print("SUJAL IS NOT HERE")
# Summarize across groups (num_sentences controls how many groups to pick)
summary_text = summarizer(all_text, num_sentences=4)  # pick top 4 important groups
print("SUJAL CODE IS WORKING")
# Map summary back to original grouped sentences to get metadata
important_groups = []
for filename, timestamp, grouped_sent in related_results:
    # Check if this grouped sentence appears in the summary
    if grouped_sent in summary_text:
        important_groups.append((filename, timestamp, grouped_sent))

# Print important groups with metadata
print("\nMost Important Groups with Metadata:")
for filename, timestamp, grouped_sent in important_groups:
    print(f"- [{filename} | {timestamp}] {grouped_sent}\n")


SUJAL IS NOT HERE


: 